# Zadanie 1

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect(r"C:\Users\klosn\Downloads\analityka.db")
cur = con.cursor()

In [3]:
base_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

cur.execute(base_query)
print(cur.fetchall())

[('events',)]


In [4]:
cur.execute("SELECT * FROM events limit 20").fetchall()

[(1642045768000, 'payment', '89', 27.4),
 (1638290115000, 'link_click', '28', None),
 (1639685873000, 'page_view', '12', None),
 (1649543869000, 'link_click', '13', None),
 (1633870039000, 'link_click', '86', None),
 (1649733293000, 'link_click', '05', None),
 (1634209485000, 'page_view', '10', None),
 (1649221589000, 'link_click', '10', None),
 (1633205236000, 'page_view', '26', None),
 (1641761469000, 'link_click', '70', None),
 (1654285475000, 'link_click', '23', None),
 (1641788363000, 'payment', '68', 32.32),
 (1653202638000, 'link_click', '30', None),
 (1649645563000, 'link_click', '40', None),
 (1636278738000, 'payment', '12', 96.81),
 (1653656739000, 'payment', '73', 9.1),
 (1655231119000, 'link_click', '08', None),
 (1649646139000, 'payment', '33', 4.44),
 (1632831836000, 'page_view', '14', None),
 (1650823165000, 'page_view', '98', None)]

In [5]:
names = list(map(lambda x: x[0], cur.description))

In [6]:
names

['event_timestamp', 'event_name', 'user_id', 'value']

### 1. Podaj łączną ilość wyświetleń strony (event_name = page_view) w przeciągu ostatnich 7 dni.


In [7]:
cur.execute("SELECT datetime(event_timestamp / 1000, 'unixepoch') from events limit 20").fetchall()

[('2022-01-13 03:49:28',),
 ('2021-11-30 16:35:15',),
 ('2021-12-16 20:17:53',),
 ('2022-04-09 22:37:49',),
 ('2021-10-10 12:47:19',),
 ('2022-04-12 03:14:53',),
 ('2021-10-14 11:04:45',),
 ('2022-04-06 05:06:29',),
 ('2021-10-02 20:07:16',),
 ('2022-01-09 20:51:09',),
 ('2022-06-03 19:44:35',),
 ('2022-01-10 04:19:23',),
 ('2022-05-22 06:57:18',),
 ('2022-04-11 02:52:43',),
 ('2021-11-07 09:52:18',),
 ('2022-05-27 13:05:39',),
 ('2022-06-14 18:25:19',),
 ('2022-04-11 03:02:19',),
 ('2021-09-28 12:23:56',),
 ('2022-04-24 17:59:25',)]

In [8]:
cur.execute("""SELECT count(*) FROM events """).fetchall()

[(1000,)]

In [9]:
cur.execute("""SELECT *, date(event_timestamp / 1000, 'unixepoch') FROM events 
            where datetime(event_timestamp / 1000, 'unixepoch') > date('now', '-7 day')
            and event_name = 'page_view'
            """).fetchall()

[(1664639251000, 'page_view', '22', None, '2022-10-01'),
 (1664671176000, 'page_view', '69', None, '2022-10-02'),
 (1664656813000, 'page_view', '60', None, '2022-10-01')]

In [10]:
views_number = len(cur.execute("""SELECT *, date(event_timestamp / 1000, 'unixepoch') FROM events 
            where datetime(event_timestamp / 1000, 'unixepoch') > date('now', '-7 day')
            and event_name = 'page_view'
            """).fetchall())

In [11]:
print(f'W ciągu ostatnich 7 dni pojawiła się następująca liczba wyświetleń strony: {views_number}.')

W ciągu ostatnich 7 dni pojawiła się następująca liczba wyświetleń strony: 3.


### 2. Podaj liczbę użytkowników strony (unikalne event_name = page_view po user_id) dla każdego z ostatnich 7 dni.

In [12]:
cur.execute("""SELECT date(event_timestamp / 1000, 'unixepoch') as date, count(distinct user_id) as cnt
            FROM events 
            where datetime(event_timestamp / 1000, 'unixepoch') > date('now', '-7 day')
            and event_name = 'page_view'
            group by 1
            """).fetchall()

[('2022-10-01', 2), ('2022-10-02', 1)]

### 3. Podaj najnowszy event dla każdego z użytkowników. Wynik tego zapytania powinien być sortowany po dacie eventu (od najnowszych) i zlimitowany do 10 wierszy.

In [13]:
cur.execute("""SELECT user_id, event_name, max(datetime(event_timestamp / 1000, 'unixepoch')) as max_date
            FROM events
            group by 1
            order by max_date desc
            limit 10
            """).fetchall()

[('84', 'link_click', '2022-10-02 19:04:10'),
 ('64', 'payment', '2022-10-02 11:23:54'),
 ('44', 'link_click', '2022-10-02 10:36:52'),
 ('72', 'payment', '2022-10-02 04:09:33'),
 ('69', 'page_view', '2022-10-02 00:39:36'),
 ('60', 'page_view', '2022-10-01 20:40:13'),
 ('95', 'link_click', '2022-10-01 20:09:36'),
 ('22', 'page_view', '2022-10-01 15:47:31'),
 ('68', 'payment', '2022-10-01 15:08:59'),
 ('41', 'link_click', '2022-10-01 08:28:18')]

# Zadanie 2

### 1. Ilość znaków w treści (type = "unstyled").
### 2. Użycie każdego znaku (type = "unstyled").

In [14]:
import json
import string

In [15]:
file = open(r'C:\Users\klosn\Downloads\article.json', encoding='utf-8')
content = json.load(file)

In [16]:
freq_dict = {strg:0 for strg in string.printable}

for dictionary in content['blocks']:
    if dictionary['type'] == 'unstyled':
        text = dictionary['text']
        print(text)
        for strg in text:
            try:
                freq_dict[strg] += 1
            except:
                print('')
                print('New letter: ', strg)
                freq_dict[strg] = 1

Podczas gdy PiS ucieka od tematu aborcji, chcąc przeczekać do wyborów, Kaja Godek i Kościół naciskają na TK, by ustawę jeszcze zaostrzył, a PO przekonuje, że obecny "kompromis" jest święty, Polki i Polacy w swoich poglądach zbliżają się do Europy. W 2018 roku swoje ustawy zliberalizowały Cypr i Irlandia. Na naszym kontynencie aborcji z powodów społecznych lub na życzenie nie dopuszcza tylko Malta, Andora, Watykan. I Polska. 

New letter:  ą

New letter:  ć

New letter:  ó

New letter:  ś

New letter:  ł

New letter:  ę

New letter:  ż
 
Sondaż pokazuje na słabnący wpływ katolickiej etyki, która od "wzmożenia moralnego" po śmierci Jana Pawła II w 2005 roku rządziła postawami. 
Od 1993 roku (z półroczna przerwą w 1997) obowiązuje w Polsce restrykcyjna ustawa, która dopuszcza przerwanie ciąży w trzech wyjątkowych przypadkach: ratowania zdrowia i życia kobiety, gdy ciąża jest wynikiem gwałtu i gdy płód jest nieuleczalnie chory (przesłanka embriopatologiczna). Legalnych aborcji jest niewiel

In [17]:
for x in list(freq_dict.keys()):
    if freq_dict[x] == 0:
        freq_dict.pop(x)

In [18]:
print(f'Wszystkich znaków jest: {sum(freq_dict.values())},\n a użycie znaków to: \n {freq_dict}')

Wszystkich znaków jest: 7665,
 a użycie znaków to: 
 {'0': 41, '1': 28, '2': 27, '3': 13, '4': 11, '5': 11, '6': 8, '7': 4, '8': 10, '9': 12, 'a': 559, 'b': 119, 'c': 273, 'd': 215, 'e': 456, 'f': 5, 'g': 67, 'h': 51, 'i': 489, 'j': 173, 'k': 179, 'l': 120, 'm': 117, 'n': 311, 'o': 519, 'p': 206, 'r': 286, 's': 229, 't': 230, 'u': 113, 'w': 287, 'y': 254, 'z': 305, 'A': 5, 'B': 4, 'C': 7, 'D': 5, 'E': 4, 'G': 4, 'I': 9, 'J': 3, 'K': 8, 'L': 3, 'M': 3, 'N': 14, 'O': 16, 'P': 38, 'R': 3, 'S': 19, 'T': 7, 'U': 1, 'W': 21, 'Z': 5, '!': 1, '"': 42, '(': 14, ')': 14, ',': 74, '-': 22, '.': 89, '/': 4, ':': 4, '=': 1, '[': 2, '\\': 1, ']': 2, ' ': 1107, 'ą': 58, 'ć': 22, 'ó': 38, 'ś': 33, 'ł': 89, 'ę': 54, 'ż': 75, 'ź': 5, 'ń': 7}


### 3. Ilość linków (entityMap -> type = "LINK").
### 4. Ilość zdjęć w tekście (entityMap -> type = "IMAGE").
### 5. Spis Id artykułów powiązanych (entityMap -> type = "RELATED_CONTENTS").

In [19]:
links_count = 0
images_count = 0
reladed_id = []

for dictionary in content['entityMap'].values():
    if dictionary['type'] == 'LINK':
        links_count += 1
    if dictionary['type'] == 'IMAGE':
        images_count += 1
    if dictionary['type'] == 'RELATED_CONTENTS':
        reladed_id.append(dictionary['data']['nodes'][0])

In [20]:
print(f'Liczba linków: {links_count}')
print(f'Liczba obrazów: {images_count}')
print(f'Id artykułów powiązanych: {reladed_id}')

Liczba linków: 13
Liczba obrazów: 10
Id artykułów powiązanych: ['213ed3dd-800d-4b9b-abab-8a99e4e3cba7', '6bed0d13-44db-40f4-8481-4b415bb8cd76']
